In [1]:
from transformers import AutoModel
import torch


# torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = AutoModel.from_pretrained("THUDM/chatglm-6b", load_in_8bit=True, trust_remote_code=True, device_map='auto')

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: Loading binary E:\dev\ChatGLM-6B\venv\lib\site-packages\bitsandbytes\libbitsandbytes_cuda116.dll...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [3]:
from peft import get_peft_model, LoraConfig, TaskType

# peft_path = "output/adapter_model.bin"
peft_path = "lora-alpaca-zh/adapter_model.bin"

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=True,
    r=16,
    lora_alpha=32, lora_dropout=0.1
)

model = get_peft_model(model, peft_config)
model.load_state_dict(torch.load(peft_path), strict=False)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [4]:
import json

# instructions = json.load(open("data/valc/valc_data.json", encoding='utf-8'))
instructions = json.load(open("data/alpaca-zh/alpaca-zh_data.json", encoding='utf-8'))

In [5]:
answers = []
from convert_json import format_example

def generate(input_text, temperature):
    ids = tokenizer.encode(input_text)
    input_ids = torch.LongTensor([ids])
    out = model.generate(
        input_ids=input_ids,
        max_length=300,
        do_sample=False,
        temperature=temperature
    )
    out_text = tokenizer.decode(out[0])
    return out_text

with torch.no_grad():
    for idx, item in enumerate(instructions[:3]):
        feature = format_example(item)
        input_text = feature['context']
        out_text = generate(input_text, 0)
        answer = out_text.replace(input_text, "").replace("\nEND", "").strip()
        item['infer_answer'] = answer
        print(out_text)
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')
        answers.append({'index': idx, **item})

E:\dev\ChatGLM-6B\venv\lib\site-packages\transformers\generation\utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
The dtype of attention mask (torch.int64) is not bool


KeyboardInterrupt: 

In [6]:
generate('你是谁', 0)

KeyboardInterrupt: 